In [23]:
import re
import sys
import pprint
import json
from IPython.display import display
from colorama import Fore, Back, Style
import subprocess

In [236]:
subprocess.run(["python3", "lexical-analyzer.py"])
with open('./results/lab1.json') as lab1_file:
    data = json.load(lab1_file)
    
tokens_chain = data['chain']
service_words = data['tables']['service_words']
operations = data['tables']['operations']
separators = data['tables']['separators']
constants = data['tables']['constants']
identifiers = data['tables']['identifiers']

nxtsymb = []
cursymb = []
token_index = -1
result = wrapper()
if result:
    error(result)

var
var_type
var
var_type
interval
interval
interval
function_or_procedure
procedure
var
var_type
var
var_type
interval
interval
var
var_type
var
var_type
interval
interval
interval
function_or_procedure
procedure
operator
assignment
expression
term
factor
argument
Anti-scan: ['C', 2, '1', 10]
Argument: 1
Anti-scan: ['C', 2, '1', 10]
Anti-scan: ['C', 2, '1', 10]
expression checked
operator
operator
assignment
expression
term
factor
argument
Anti-scan: ['I', 2, 'arg2', 13]
Argument: arg2
Anti-scan: ['I', 2, 'arg2', 13]
Anti-scan: ['I', 2, 'arg2', 13]
expression checked
operator
Fatal: unexpected "return" at line 14
Fatal: invalid procedure declaration at line 14
Fatal: invalid function or procedure declaration at line 14


In [220]:
def scan():
    global tokens_chain, token_index, nxtsymb, cursymb
    token_index += 1
    if token_index < len(tokens_chain):
        cursymb = tokens_chain[token_index-1]
        nxtsymb = tokens_chain[token_index]

In [126]:
def antiscan():
    global tokens_chain, token_index, nxtsymb, cursymb
    token_index -= 1
    if token_index >= 1:
        cursymb = tokens_chain[token_index-1]
        nxtsymb = tokens_chain[token_index]

In [229]:
def error(text):
    global cursymb
    if text[-8:] == 'expected':
        text += ' instead of "' + nxtsymb[2] + '"'
    print(Fore.RED + 'Fatal:', Fore.BLACK + text, Style.RESET_ALL + Fore.WHITE + 'at line', Style.RESET_ALL + str(nxtsymb[3]))

In [214]:
def check(response):
    if response:
        error(response)
        return False
    else:
        return True

In [207]:
def wrapper():
    global nxtsymb, cursymb
    scan()
    if nxtsymb[2] == 'program':
        if not check(program()):
            return 'invalid program declaration'
        
        scan()
    if nxtsymb[2] == 'var':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        if nxtsymb[2] != ';':
            return '";" expected'
        scan()
        while nxtsymb[0] == 'I':
            if not check(var()):
                return 'invalid var declaration'
            scan()
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
    
    while nxtsymb[2] in ['function', 'procedure']:
        if not check(function_or_procedure()):
            return 'invalid function or procedure declaration'
    
    if nxtsymb[2] != 'begin':
        return '"begin" expected'
    scan()
    
    mistake = ''
    while not mistake:
        mistake = operator()
        if mistake:
            check(mistake)
        if cursymb[0] == 'I' and nxtsymb[2] == ':':
            scan()
        else:
            scan()
            print('_____---->', cursymb, nxtsymb)
            if nxtsymb[2] == ';':
                scan()
                if nxtsymb[2] == 'end.':
                    print('End of program')
                    break
            else:
                return '";" expected' 

In [7]:
def program():
    global nxtsymb
    print(Fore.GREEN + 'program' + Style.RESET_ALL)
    
    scan()
    if nxtsymb[0] != 'I':
        return 'identifier expected'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    return ''

In [8]:
def var():
    global nxtsymb
    print(Fore.GREEN + 'var' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'I':
        return 'identifier expexted'
    scan()
    while nxtsymb[2] == ',':
        scan()
        if nxtsymb[0] != 'I':
            return 'identifier expected'
        scan()
    if nxtsymb[2] != ':':
        return '":" expected'
    scan()
    if not check(var_type()):
        return 'invalid variable type declaration'
    return ''

In [9]:
def var_type():
    global nxtsymb
    print(Fore.GREEN + 'var_type' + Style.RESET_ALL)
    
    if nxtsymb[2] in ['integer', 'real', 'string']:
        return ''
    elif nxtsymb[2] == 'array':
        scan()
        if nxtsymb[2] != '[':
            return '"[" expected'
        scan()
        if not check(interval()):
            return 'invalid interval syntax'
        scan()
        while nxtsymb[2] == ',':
            scan()
            if not check(interval()):
                return 'invalid interval syntax'
            scan()
        if nxtsymb[2] != ']':
            return '"]" expected'
        scan()
        if nxtsymb[2] != 'of':
            return '"of" expected'
        scan()
        if nxtsymb[2] not in ['integer', 'real', 'string']:
            return 'variable type expected'
        return ''
    else:
        return 'array type expected'
        
            

In [10]:
def interval():
    global nxtsymb
    print(Fore.GREEN + 'interval' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'C':
        return 'constant expected'
    scan()
    if nxtsymb[2] != '..':
        return '".." expected'
    scan()
    if nxtsymb[0] != 'C':
        return 'constant expected'
    return ''

In [11]:
def function_or_procedure():
    global nxtsymb
    print(Fore.GREEN + 'function_or_procedure' + Style.RESET_ALL)
    
    if nxtsymb[2] == 'procedure':
        scan()
        if not check(procedure()):
            return 'invalid procedure declaration'
        return ''
    if nxtsymb[2] == 'function':
        scan()
        if not check(function()):
            return 'invalid function declaration'
        return ''
    return ''        

In [12]:
def procedure():
    global nxtsymb
    print(Fore.GREEN + 'procedure' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'I':
        return 'identifier expended'
    scan()
    if nxtsymb[2] == '(':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        while nxtsymb[2] == ';':
            scan()
            if not check(var()):
                return 'invalid var declaration'
            scan()
        if nxtsymb[2] != ')':
            return '")" expected'
        scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    
    scan()
    if nxtsymb[2] == 'var':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        if nxtsymb[2] != ';':
            return '";" expected'
        scan()
        while nxtsymb[0] == 'I':
            if not check(var()):
                return 'invalid var declaration'
            scan()
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
    
    while nxtsymb[2] in ['function', 'procedure']:
        if not check(function_or_procedure()):
            return 'invalid function or procedure declaration'
    
#     scan()
    if nxtsymb[2] != 'begin':
        return '"begin" expected'
    scan()
    while len(operator()) == 0:
        scan()
        if cursymb[2] != ':':
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
    if nxtsymb[2] != 'end':
        if cursymb[2] == ';' and nxtsymb[2] == 'else':
            return 'extra ";" before else'
        return 'unexpected "'+nxtsymb[2]+'"'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    scan()
    return ''

In [209]:
def function():
    global nxtsymb
    print(Fore.GREEN + 'function' + Style.RESET_ALL)
    
    if nxtsymb[0] != 'I':
        return 'identifier expended'
    scan()
    if nxtsymb[2] == '(':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        while nxtsymb[2] == ';':
            scan()
            if not check(var()):
                return 'invalid var declaration'
            scan()
        if nxtsymb[2] != ')':
            return '")" expected'
        scan()
    if nxtsymb[2] != ':':
        return 'expected ":"'
    scan()
    if nxtsymb[2] not in ['integer', 'real', 'string']:
        return 'type of function expected'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    
    scan()
    if nxtsymb[2] == 'var':
        scan()
        if not check(var()):
            return 'invalid var declaration'
        scan()
        if nxtsymb[2] != ';':
            return '";" expected'
        scan()
        while nxtsymb[0] == 'I':
            if not check(var()):
                return 'invalid var declaration'
            scan()
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
    
    while nxtsymb[2] in ['function', 'procedure']:
        if not check(function_or_procedure()):
            return 'invalid function or procedure declaration'
    
#     scan()
    if nxtsymb[2] != 'begin':
        return '"begin" expected'
    
    scan()
    while len(operator()) == 0:
        scan()
        if cursymb[2] != ':':
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
    
    if nxtsymb[2] != 'return':
        return 'function have to return some value'
    scan()
    if not check(expression()):
        return 'return\'s exression expected'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    
    scan()
    while len(operator(True)) == 0:
        scan()
        if cursymb[2] != ':':
            if nxtsymb[2] != ';':
                return '";" expected'
            scan()
            
    if nxtsymb[2] != 'end':
        if cursymb[2] == ';' and nxtsymb[2] == 'else':
            return 'extra ";" before else'
        return 'unexpected "'+nxtsymb[2]+'"'
    scan()
    if nxtsymb[2] != ';':
        return '";" expected'
    scan()
    return ''

In [210]:
def operator(and_return = False):
    global nxtsymb
    print(Fore.GREEN + 'operator' + Style.RESET_ALL)

    if nxtsymb[2] == ';':
        pass
    elif nxtsymb[0] == 'I':
        scan()
        if nxtsymb[2] == ':':
            return ''
        if nxtsymb[2] == '[':
            scan()
            if not check(expression()):
                return 'invalid array arguments'
            scan()
            while nxtsymb[2] == ',':
                scan()
                if not check(expression()):
                    return 'invalid array arguments'
                scan()
            if nxtsymb[2] != ']':
                return '"]" expected'
            scan()
        if nxtsymb[2] == ':=':
            if not check(assignment()):
                return 'invalid assignment declaration'
            return ''
        if nxtsymb[2] == '(':
            scan()
            if not check(expression()):
                return 'invalid argument'
            scan()
            while nxtsymb[2] == ',':
                scan()
                if not check(expression()):
                    return 'invalid argument'
                scan()
            if nxtsymb[2] != ')':
                return 'expected ")"'
            return ''
        return 'unexpected "'+nxtsymb[2]+'"'
    elif nxtsymb[2] == 'if':
        scan()
        if not check(condition()):
            return 'condition expected'
        scan()
        if nxtsymb[2] != 'then':
            return '"then" expected'
        scan()
        if not check(operator()):
            return 'operator expected'
        scan()
        if nxtsymb[2] == 'else':
            scan()
            if not check(operator()):
                return 'operator expected'
            return ''
        elif nxtsymb[2] == ';':
            scan()
            if nxtsymb[2] == 'else':
                antiscan()
                return 'extra ";" before "else"'
            antiscan()
        antiscan()
        return ''
    elif nxtsymb[2] == 'goto':
        scan()
        if nxtsymb[0] != 'I':
            return 'identifier expected'
        return ''
    elif nxtsymb[2] == 'while':
        scan()
        if not check(condition()):
            return 'condition expected'
        scan()
        if nxtsymb[2] != 'do':
            return '"do" expected'
        scan()
        if not check(operator()):
            return 'operator expected'
    elif and_return and nxtsymb[2] == 'return':
        scan()
        if not check(expression()):
            return 'return\'s exression expected'
        return ''
    else:
        return 'operator expected'
    return ''

In [16]:
def condition():
    global nxtsymb
    print(Fore.GREEN + 'condition' + Style.RESET_ALL)
    
    if not check(expression()):
        return 'expression expected'
    scan()
    if nxtsymb[2] not in ['=', '>', '<', '>=', '<=', '<>']:
        return 'comparasion operation'
    scan()
    if not check(expression()):
        return 'expression expected'
    return ''

In [17]:
def assignment():
    global nxtsymb
    print(Fore.GREEN + 'assignment' + Style.RESET_ALL)
    
    if nxtsymb[2] != ':=':
        return '":=" expected'
    scan()
    if not check(expression()):
        return 'invalid expression'
    print(Fore.GREEN + 'expression checked' + Style.RESET_ALL)
    return ''

In [18]:
def expression():
    print(Fore.GREEN + 'expression' + Style.RESET_ALL)
    global nxtsymb
    
#     print('Terms:')
    if not check(term()):
        return 'term expected'
#     print(Fore.BLUE + 'Term:' + Style.RESET_ALL, nxtsymb[2])
    scan()
    while nxtsymb[2] in ['+', '-']:
        scan()
        if not check(term()):
            return 'term expected'
#         print(Fore.BLUE + 'Term:' + Style.RESET_ALL, nxtsymb[2])
        scan()
    else:
        antiscan()
        pass
        
    return ''

In [19]:
def term():
    global nxtsymb
    print(Fore.GREEN + 'term' + Style.RESET_ALL)
    
    if not check(factor()):
        return 'factor expected'
#     print(Fore.BLUE + 'Factor:' + Style.RESET_ALL, nxtsymb[2])
    scan()
    while nxtsymb[2] in ['*', '/', '^']:
        scan()
        if not check(factor()):
            return 'factor expected'
#         print(Fore.BLUE + 'Factor:' + Style.RESET_ALL, nxtsymb[2])
        scan()
    else:
        antiscan()
        pass
#         print('End of factors')
        
    return ''

In [20]:
def factor():
    global nxtsymb
    print(Fore.GREEN + 'factor' + Style.RESET_ALL)
    
    if nxtsymb[2] == '(':
        scan()
        if not check(expression()):
            return 'expression expected'
        scan()
        if nxtsymb[2] != ')':
            return '")" expected'
    else:
        if not check(argument()):
            return 'constant or identifier expected'
#         print('heh')
    return ''

In [21]:
def argument():
    global nxtsymb
    print(Fore.GREEN + 'argument' + Style.RESET_ALL)

    if nxtsymb[0] not in ['I', 'C']:
        return 'constant or identifier expected'
    scan()
    if nxtsymb[2] in ['[', '(']:
        scan()
        if not(check(expression())):
            return 'invalid expression'
        scan()
        while nxtsymb[2] == ',':
            scan()
            if not(check(expression())):
                return 'invalid expression'
            scan()
        if nxtsymb[2] not in [']', ')']:
            return 'expected \']\' or \')\''
    else:
        antiscan()
    print(Fore.BLUE + 'Argument:' + Style.RESET_ALL, nxtsymb[2])
    return ''